## A novel approach to Plasmid assembly

#### Contents:
Novel 

In [1]:
# Get directory
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import subprocess
import shutil
RUN_DIR = os.getenv("RUN_DIR")
PRESENTATION_DIR = os.getenv("PRESENTATION_DIR")

In [2]:
# Create sample class
class Sample:
    """The sample class is used to define all the files used in the pipeline.
       Both Inputs and Outputs. We can use this class object to run our pipeline multiple times
       with different setting without having to write multiple sets of code."""
    def __init__(self, barcode, name, description, fastq_file):
        self.name = name
        self.barcode = barcode
        self.description=description
        # Get raw data (fastq files)
        self.fastq_path = os.path.join(SUBDIRS["fastq"], fastq_file)
        self.fastq_file = fastq_file
        # Set pauvre plot output file
        self.pauvre_output_plot = os.path.join(SUBDIRS["pauvre"], self.fastq_file.replace(".fastq", ".png"))
        # Set trimmed files
        self.fastq_file_trimmed = os.path.join(SUBDIRS["trimmed"], self.name+".trimmed.fastq")
        self.trimmed_logs = os.path.join(SUBDIRS["trimmed"], self.name+".trimmed.log")
        # Canu related files
        self.est_genome_length = 0
        self.canu_log_output = os.path.join(SUBDIRS["canu"], self.sample, self.name+".stderr.log")
        self.canu_tig_info_file = os.path.join(SUBDIRS["canu"], self.sample, self.name+".contigs.layout.tigInfo")
        self.canu_contig_file = os.path.join(SUBDIRS["canu"], self.sample, self.name+".contigs.fasta")
        self.canu_trimmed_corrected_reads = os.path.join(SUBDIRS["canu"], self.sample, self.name+".trimmedReads.fasta.gz")
        self.canu_unassembled_fasta_file = os.path.join(SUBDIRS["canu"], self.sample, self.name+".unassembled.fasta."
        # Circlator related files
        self.circlator_output_prefix = os.path.join(SUBDIRS["circlator"], self.sample, self.sample)
        self.circlator_output_fasta = os.path.join(SUBDIRS["circlator"], self.sample, self.sample+".circularise.fasta")
        # Alignment related files
        self.sam_file = os.path.join(SUBDIRS["alignment"], self.sample, self.sample+".sam")
        self.bam_file = os.path.join(SUBDIRS["alignment"], self.sample, self.sample+".bam")
        self.bai_file = os.path.join(SUBDIRS["alignment"], self.sample, self.sample+".bai")
        # MiSeq data for hybrid assemblies
        self.short_read_dir = os.path.join(SUBDIRS["illumina_data"], self.sample)
        self.short_read_fastq1_file = [fastq_file for fastq_file in os.listdir(self.short_read_dir)
                                       if fastq_file.endwith("_R1.fastq")][0]
        self.short_read_fastq2_file = [fastq_file for fastq_file in os.listdir(self.short_read_dir)
                                       if fastq_file.endwith("_R2.fastq")][0]
        # Unicycler related files
        self.unicycler_directory = os.path.join(SUBDIRS["hybrid_assemblies"], self.sample)
        self.unicycler_assembly_file = os.path.join(self.unicycler_directory, "assembly.fasta")

/data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED


In [ ]:
"""In this code chunk we define all of the commands.
   We could do this in the sample section, and create each function as an a attribute of an object in the sample class.
   ... but I feel this is a little too abstract and ruins the flow of the script.
   Not all commands will be here, there will be some cheeky manipulations occuring in the main script.
   Particularly stuff that is exported to csvs for purpose of powerpoint presentation."""

# First up we have the pauvre command
def run_pauvre(input_fastq_file, title, output_folder):
    pauvre_command_options=["python3 $(which pauvre)"]
    pauvre_command_options.append("marginplot")
    pauvre_command_options.append("--fastq %s" % input_fastq_file)
    pauvre_command_options.append("--title 'Distribution of %s'" % title)
    pauvre_command = ' '.join(pauvre_command_options)
    pauvre_proc = subprocess.Popen(pauvre_command, shell=True,
                                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = pauvre_proc.communicate()
    # Pauvre file created in the current working directory. Need to move to approrpriate directory
    output_file = os.path.basename(input_fastq_file).replace(".fastq", ".png")
    shutil.move(output_file, output_folder)
    
# Next the porechop command
def run_porechop(fastq_file, output_fastq_file, output_log_file):
    porechop_command_options = ["porechop"]
    porechop_command_options.append("--input %s" % fastq_file)
    porechop_command_options.append("--discard_middle")  # Remove any reads with adapters in the middle.
    porechop_command_options.append("--require_two_barcodes")  # Remove any reads with adapters in the middle.
    porechop_command_options.append("--output %s" % output_fastq_file)
    porechop_command_options.append("2> %s" % output_log_file)
    porechop_command = ' '.join(porechop_command_options)
    # Run through subprocess
    porechop_proc = subprocess.Popen(porechop_command, shell=True,
                                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = porechop_proc.communicate()
    print(stdout, stderr)

# Canu
def run_canu(prefix, directory, estimated_genome_length, input_fastq_file, canu_stderr_log)
    canu_command_options = ["canu"]
    canu_command_options.append("-p %s" % sample)
    canu_command_options.append("-d %s" % sample_canu_dir)
    canu_command_options.append("genomeSize=%d" % est_genome_length)
    canu_command_options.append("useGrid=false")
    canu_command_options.append('stopOnReadQuality=false')
    canu_command_options.append("-nanopore-raw")
    canu_command_options.append("%s.all.trimmed.fastq" % os.path.join(trimmed_dir, sample))
    canu_command_options.append("2> %s" % os.path.join(sample_canu_dir, sample + ".stderr.log"))
    canu_command = ' '.join(canu_command_options)

    canu_proc = subprocess.Popen(canu_command, shell=True,
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = canu_proc.communicate()
    print(stdout, stderr)
    
# Circlator
def run_circlator(sample_name, input_contig_file, output_contig_prefix):
    if os.path.isdir(circlator_sample_dir):
    shutil.rmtree(circlator_sample_dir)
    os.mkdir(circlator_sample_dir) 
    # Circlator command
    circlator_command_options = ["circlator minimus2"]
    circlator_command_options.append(input_contig_file)
    circlator_command_options.append(output_contig_prefix)
    circlator_command = ' '.join(circlator_command_options)
    # Remove 
    sed_command = "sed -i \"1 s/^>tig0000000\\d/>%s.circularised/\" %s" % (sample_name, circlator_output_prefix+".circularise.fasta")
    # Run commands sequentially
    circlator_proc = subprocess.Popen(' && '.join([circlator_command, sed_command]), shell=True,
                                                   stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Sam2bamalignment
def run_bwamem_sam2bam(reference, input_fastq_file, sam_file, bam_file, bai_file):
    # Create the bwa and samtools indexes for the draft reference
    bwa_index_command = "bwa index %s" % reference
    samtools_index_command = "samtools faidx %s" % reference
    index_proc = subprocess.Popen(' && '.join([bwa_index_command, samtools_index_command]), shell=True,
                                  stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = index_proc.communicate()
    print(stdout, stderr)
    
    # Now run the bwa and sam2bam command
    bwa_align_command = "bwa mem -x ont2d %s %s > %s" % (reference, input_fastq_file, sam_file)
    
    # Samtools view and sort command
    samtools_view_and_sort_command = "samtools view -bS %s | samtools sort -o %s -" % (sam_file, bam_file)
    
    # Create bam index
    bam_index_command = "samtools index %s %s" % (bam_file, bai_file)
    
    # Run all commands sequentially
    alignment_commands = ' && '.join([bwa_align_command, samtools_view_and_sort_command,
                                       bam_index_command])
    alignment_proc = subprocess.Popen(alignment_commands, shell=True,
                                      stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = alignment_proc.communicate()

In [ ]:
os.chdir(RUN_DIR)
subdirs_list = ["fastq", "pauvre", "trimmed", "canu", "circlator", "alignment", "illumina_data"]
SUBDIRS = {key,os.path.join(RUN_DIR, value) for key,value in itertools.izip(subdirs_list, subdirs_list)}
SAMPLES = []  # List of objects of the class SAMPLE
barcode2sample_dict = {"barcode01": "JB2",
                       "barcode02": "JB1",
                       "barcode11": "PTS1",
                       "barcode12": "PTS2",
                       "barcode06": "YHC6",
                       "barcode07": "YHC7",
                       "barcode08": "YHC8",
                       "barcode09": "YHC17",
                       "unclassified": "unclassified"}

fastq_files = [fastq_file for fastq_file in os.listdir(SUBDIRS["fastq"])
               if fastq_file.startswith("barcode")]

# Add barcodes to the sample class
for fastq_file in fastq_files:
    # fastq_file barcode07.all.fastq
    barcode = fastq_file.split(".")[0]
    name = barcode2sample_dict[barcode]
    SAMPLES.append(Sample(barcode=barcode, name=name. description=None, fastq_file=fastq_file))

#### Presentation orientated code_chunk

In [3]:
### Function to count the number of reads per fastq file.
### Output to csv with the following columns:
### Bin, barcode, num nucleotides per barcode/bin
csv_output_file = os.path.join(PRESENTATION_DIR, "data", "size_by_barcode.tsv")
bin_df = pd.DataFrame(columns=["bin", "sample", "nucleotides"])
for fastq_file in fastq_files:
    # 0005_49335_plasmids.barcode07.0.fastq
    bin_id = fastq_file.split("_")[0]
    barcode = fastq_file.split(".")[1]
    # Get number of nucleotides
    num_nucleotides = 0
    with open(os.path.join(SUBDIRS["fastq"], fastq_file), "rU") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            num_nucleotides += len(record.seq)
    df_as_series = pd.Series(data=[bin_id, barcode2sample_dict[barcode], num_nucleotides], index=["bin", "sample", "nucleotides"])
    bin_df = bin_df.append(df_as_series, ignore_index=True)
bin_df.to_csv(csv_output_file, index=False, sep="\t")
# Get minimum sample we will move the pauvre plot of this sample to the PRESENTATION_DIR
grouped = bin_df.groupby("sample")
min_sample_name = grouped['nucleotides'].aggregate(np.sum).idxmin()

#### Now run through qc and assembly pipeline

In [ ]:
# Run the pauvre command
for sample in SAMPLES:
    run_pauvre(input_fastq_file=sample.fastq_path, 
               title=sample.name, 
               output_folder=SUBDIRS["pauvre"])
    
# Copy pauvre plot of smallest file to PRESENTATION_DIR
pauvre_min_output_plot = os.path.join(SUBDIRS["pauvre"], min_sample_name+".all.png")
pauvre_plot_presentation_file = os.path.join(PRESENTATION_DIR, "images", "pauvre_min.png")
shutil.copy(pauvre_min_output_plot, pauvre_plot_presentation_file)
                                      
# Use porechop to trim adapters off reads
for sample in SAMPLES:
    run_porechop(fastq_file=sample.fastq_path, 
                 output_fastq_file=sample.fastq_file_trimmed, 
                 output_log_file=sample.trimmed_logs)

#### We also need to estimate the genome lengths before we continue
As Canu requires this as an input parameter

In [7]:
# Estimate genome size using top five percent of reads.
for sample in SAMPLES:
    with open(sample.fastq_file_trimmed, "rU") as handle:
        record_dict = SeqIO.to_dict(SeqIO.parse(handle, "fastq"))
    number_reads = len(record_dict)
    iterator = 0
    number_reads_top5 = number_reads*0.05
    cumulative_sum = 0
    read_list = []
    for id in sorted(record_dict, key=lambda id: len(record_dict[id].seq), reverse=True):
        iterator += 1
        cumulative_sum += len(record_dict[id].seq)
        read_list.append(len(record_dict[id].seq))
        if iterator > number_reads_top5:
            break
            
    mean = cumulative_sum/iterator
    print("%s: top 5 mean: %d. Over %d reads." % (fastq_file, mean, iterator))
    sample.est_genome_length = mean

YHC17.all.trimmed.fastq
YHC17.all.trimmed.fastq: top 5 mean: 21705. Over 10 reads. reads are: 26923 22132 22090 22090 21669 21557 21060 20027 19884 19620
YHC6.all.trimmed.fastq
YHC6.all.trimmed.fastq: top 5 mean: 20112. Over 4 reads. reads are: 37138 14579 14408 14326
YHC7.all.trimmed.fastq
YHC7.all.trimmed.fastq: top 5 mean: 15924. Over 8 reads. reads are: 19173 18436 16460 15427 15298 15181 14069 13351
JB1.all.trimmed.fastq
JB1.all.trimmed.fastq: top 5 mean: 8894. Over 219 reads. reads are: 22273 21469 16487 15694 15277 13740 13286 13019 12166 12154 11921 11247 11232 10685 9669 9309 9150 9040 8985 8921 8877 8874 8843 8836 8817 8800 8795 8743 8736 8717 8707 8700 8700 8686 8678 8663 8642 8640 8635 8624 8607 8604 8588 8584 8583 8580 8580 8580 8579 8577 8573 8572 8566 8564 8563 8562 8560 8560 8556 8555 8553 8553 8549 8549 8547 8547 8547 8546 8546 8545 8543 8542 8540 8540 8540 8539 8538 8536 8535 8533 8533 8532 8531 8527 8526 8526 8526 8523 8522 8521 8520 8519 8518 8516 8515 8515 8512 851

In [ ]:
# Use Canu to assemble genome
for sample in SAMPLES:
    run_canu(prefix=sample.name,
             directory=os.path.join(SUBDIRS["canu"], sample.name)
             estimated_genome_length=sample.est_genome_length
             input_fastq_file=sample.fastq_file_trimmed
             canu_stderr_log=sample.canu_log_output
            )

In [9]:
# Rename the first tigID for each sample
for sample in SAMPLES:
    if os.stat(sample.canu_contig_file).st_size == 0:
        # Failed to form contig, use tigInfoFile to get main contig out of unassembled file
        # Open up unassembled fasta file:
        records = SeqIO.parse(sample.canu_unassembled_fasta_file, "fasta")
        # Open up tigInfo File
        tigInfoFile = os.path.join(canu_dir, sample, sample+".contigs.layout.tigInfo")
        tigInfoDF = pd.read_csv(tigInfoFile, sep="\t", header=0)
        # Get contigs with coverage > 1
        covered_contigs = ["tig{:08d}".format(contig)
                           for contig in tigInfoDF.loc[tigInfoDF["coverage"] > 1]["#tigID"].tolist()]
        # write contigs with coverage > 1 to canu contig file
        SeqIO.write([record for record in records if record.id in covered_contigs],
                     sample.canu_contig_file, "fasta")        

In [ ]:
# Run circlator minimus2 to circularise genome
for sample in SAMPLES:
    run_circlator(input_contig_file=sample.canu_contig_file
                  output_contig_prefix=sample.circlator_output_prefix)

In [ ]:
# Align trimmed canu reads back to circulator, we can then look at this in IGV.
for sample in SAMPLES:
    run_bwamem_sam2bam(reference=sample.circulator_output_fasta,
                       input_fastq_file=sample.canu_trimmed_corrected_reads,
                       sam_file=sample.sam_file,
                       bam_file=sample.bam_file,
                       bai_file=sample.bai_file)

### Some more exporting to the PRESENTATION DIRECTORY
That's the end of the pipeline folks!
But we're not done yet.
We will export the 'circularisation' status to the presentation directory.
Then we will try add the unclassified reads back in by mapping them back to the genome.

In [32]:
# Test for circularisation, export to TSV
samples = genome_lengths.iterkeys()
circlator_dir = os.path.join(RUN_DIR, "circlator")
circularised_df = pd.DataFrame(columns=["sample", "genome_size", "circularised"])
tsv_output_file = os.path.join(PRESENTATION_DIR, "data", "genome_status.tsv")
for sample in samples:
    circlator_sample_dir = os.path.join(circlator_dir, sample)
    circlator_log_file = os.path.join(circlator_sample_dir, sample+".log")
    # Example line in log file: tig00000001 circularised: True
    is_circularised_command = "grep circularised: %s | rev | cut -d' ' -f1 | rev" % circlator_log_file
    is_circularised_proc = subprocess.Popen(is_circularised_command, shell=True,
                                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = is_circularised_proc.communicate()
    is_circularised = stdout.rstrip()=="True"
    circularised_as_seres = pd.Series(data=[sample, genome_lengths[sample], is_circularised], 
                                      index=["sample", "genome_size", "circularised"])
    circularised_df = circularised_df.append(circularised_as_seres, ignore_index=True)
circularised_df.to_csv(tsv_output_file, index=False, sep="\t")
print(circularised_df)

False
True
True
True
True
False
True
False
  sample genome_size circularised
0   YHC8       27458        False
1    JB2       12228         True
2    JB1        8894         True
3   PTS1       13987         True
4   PTS2       13553         True
5   YHC6       20112        False
6   YHC7       15924         True
7  YHC17       21705        False


In [ ]:
# This block here aligns the unclassified reads to the list of circularised genomes and tries to see
# if any of this unclassified data can be used

In [31]:
# Re-align reads using samtools
# Create alignment dir
samples = genome_lengths.iterkeys()
alignment_dir = os.path.join(RUN_DIR, "alignment")
if not os.path.isdir(alignment_dir):
    os.mkdir(alignment_dir)
# Run alignment for each sample reads    
for sample in samples:
    # Get trimmed canu reads
    sample_canu_dir = os.path.join(canu_dir, sample)
    sample_alignment_dir = os.path.join(alignment_dir, sample)
    if os.path.isdir(sample_alignment_dir):
        shutil.rmtree(sample_alignment_dir)
    os.mkdir(sample_alignment_dir)
    
    circlator_sample_dir = os.path.join(circlator_dir, sample)
    circlator_contig_file = os.path.join(circlator_sample_dir, sample+".circularise.fasta")

    sam_file = os.path.join(sample_alignment_dir, sample+".sam")
    bam_file = os.path.join(sample_alignment_dir, sample+".bam")
    bai_file = os.path.join(sample_alignment_dir, sample+".bai")

    if not os.path.isdir(sample_alignment_dir):
        os.mkdir(sample_alignment_dir)
    # Use the corrected and trimmed reads from Canu
    trimmed_reads = os.path.join(sample_canu_dir, sample+".trimmedReads.fasta.gz")
    
    # Create the bwa and samtools indexes for the draft reference
    bwa_index_command = "bwa index %s" % circlator_contig_file
    samtools_index_command = "samtools faidx %s" % circlator_contig_file
    index_proc = subprocess.Popen(' && '.join([bwa_index_command, samtools_index_command]), shell=True,
                                  stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = index_proc.communicate()
    print(stdout, stderr)
    
    # Now run the bwa and sam2bam command
    bwa_align_command = "bwa mem -x ont2d %s %s > %s" % (circlator_contig_file, trimmed_reads, sam_file)
    
    # Samtools view and sort command
    samtools_view_and_sort_command = "samtools view -bS %s | samtools sort -o %s -" % (sam_file, bam_file)
    
    # Create bam index
    bam_index_command = "samtools index %s %s" % (bam_file, bai_file)
    
    # Run all commands sequentially
    alignment_commands = ' && '.join([bwa_align_command, samtools_view_and_sort_command,
                                       bam_index_command])
    alignment_proc = subprocess.Popen(alignment_commands, shell=True,
                                      stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = alignment_proc.communicate()
    print(stdout, stderr)

('', '[bwa_index] Pack FASTA... 0.00 sec\n[bwa_index] Construct BWT for the packed sequence...\n[bwa_index] 0.00 seconds elapse.\n[bwa_index] Update BWT... 0.00 sec\n[bwa_index] Pack forward-only FASTA... 0.00 sec\n[bwa_index] Construct SA from BWT and Occ... 0.01 sec\n[main] Version: 0.7.15-r1140\n[main] CMD: bwa index /data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED/circlator/YHC8/YHC8.circularise.fasta\n[main] Real time: 0.022 sec; CPU: 0.019 sec\n')
('', '[M::bwa_idx_load_from_disk] read 0 ALT contigs\n[M::process] read 69 sequences (516009 bp)...\n[M::mem_process_seqs] Processed 69 reads in 3.809 CPU sec, 3.809 real sec\n[main] Version: 0.7.15-r1140\n[main] CMD: bwa mem -x ont2d /data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED/circlator/YHC8/YHC8.circularise.fasta /data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED/canu/YHC8/YHC8.trimmedReads.fasta.gz\n[main] Real time: 3.828 sec; CPU: 3.828 sec\n')
('', '[bwa_index] Pack FASTA... 0.00 sec\n[

In [36]:
# Use the unassembled data and map to the draft genomes
# Step 1: Create concatenated draft genome
samples = genome_lengths.iterkeys()
random_dir = os.path.join(RUN_DIR, "unclassified")
if not os.path.isdir(random_dir):
    os.mkdir(random_dir)
    
# Concatenate references into all_genomes_files
all_draft_genomes_file = os.path.join(random_dir, "all_draft_genomes.fna")
if os.path.isfile(all_draft_genomes_file):
    os.remove(all_draft_genomes_file)
for sample in samples:
    # Concatenate references into all_genomes_files
    sample_alignment_dir = os.path.join(alignment_dir, sample) 
    circlator_sample_dir = os.path.join(circlator_dir, sample)
    circlator_contig_file = os.path.join(circlator_sample_dir, sample+".circularise.fasta")
    cat_command = "cat %s >> %s" % (circlator_contig_file, all_draft_genomes_file)
    cat_proc = subprocess.Popen(cat_command, shell=True,
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = cat_proc.communicate()
    print(stdout, stderr)
    
# Create bwa and faidx
bwa_index_command = "bwa index %s" % all_draft_genomes_file
samtools_index_command = "samtools faidx %s" % all_draft_genomes_file
index_proc = subprocess.Popen(' && '.join([bwa_index_command, samtools_index_command]), shell=True,
                              stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = index_proc.communicate()

# Align all unclassified reads to the genome sets.
all_unclassified_fasta_files = os.path.join(FASTQ_DIR, "unclassified.all.fastq")
all_unclassified_trimmed_fasta_file = os.path.join(trimmed_dir, "unclassified.all.trimmed.fastq")
sam_file = os.path.join(random_dir, "unclassified.all.sam")
bam_file = os.path.join(random_dir, "unclassified.all.bam")
bai_file = os.path.join(random_dir, "unclassified.all.bai")

# But first we trim.
porechop_command_options = ["porechop"]
porechop_command_options.append("--input %s" % all_unclassified_fasta_files)
porechop_command_options.append("--discard_middle")  # Remove any reads with adapters in the middle.
porechop_command_options.append("--output %s" % os.path.join(trimmed_dir, all_unclassified_trimmed_fasta_file))
porechop_command = ' '.join(porechop_command_options)
porechop_proc = subprocess.Popen(porechop_command, shell=True,
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = porechop_proc.communicate()

# Now run the bwa and sam2bam command
bwa_align_command = "bwa mem -x ont2d %s %s > %s" % (all_draft_genomes_file, all_unclassified_trimmed_fasta_file, sam_file)

# Samtools view and sort command
samtools_view_and_sort_command = "samtools view -bS %s | samtools sort -o %s -" % (sam_file, bam_file)
    
# Create bam index
bam_index_command = "samtools index %s %s" % (bam_file, bai_file)
# Split bam file by output
bamtools_split_command = "bamtools split -in %s -reference" % bam_file

# Run all commands sequentially
alignment_commands = ' && '.join([bwa_align_command, samtools_view_and_sort_command,
                                  bam_index_command, bamtools_split_command])
alignment_proc = subprocess.Popen(alignment_commands, shell=True,
                                  stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = alignment_proc.communicate()

('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')


In [60]:
# Step 2: Convert these output bam files to fastq
bam_files = [bam_file for bam_file in os.listdir(random_dir)
             if bam_file.endswith(".bam") and
             "unmapped" not in bam_file and
             "REF" in bam_file]

# Create dictionary of bam_files
bam_files_by_sample = {}
for bam_file in bam_files:
    # Bam file: unclassified.all.REF_PTS2.circularised.bam
    print(bam_file)
    sample = bam_file.split(".")[2].split("_")[1]
    bam_files_by_sample[sample] = os.path.join(random_dir, bam_file)

for sample, bam_file in bam_files_by_sample.iteritems():
    # Run bam2fastq
    fastq_file = os.path.join(random_dir, "unclassified.%s.fastq" % sample)
    bam2fastq_command = "samtools bam2fq -O %s | seqtk seq > %s" % (bam_file, fastq_file)
    
    # Create concatenated fastq file that appends reads from trimmed directory
    original_trimmed_file = os.path.join(trimmed_dir, sample+".all.trimmed.fastq")
    
    cat_command = "cat %s >> %s" % (original_trimmed_file, fastq_file)
    
    # Run this again through canu
    sample_canu_dir = os.path.join(random_dir, "canu_%s" % sample)
    canu_prefix = "retry_canu_%s" % sample
    if os.path.isdir(sample_canu_dir):
        shutil.rmtree(sample_canu_dir)
    os.mkdir(sample_canu_dir)
    
    canu_command_options = ["canu"]
    canu_command_options.append("-p %s" % canu_prefix)
    canu_command_options.append("-d %s" % sample_canu_dir)
    canu_command_options.append("genomeSize=%d" % genome_lengths[sample])
    canu_command_options.append("useGrid=false")
    canu_command_options.append('stopOnReadQuality=false')
    canu_command_options.append("-nanopore-raw")
    canu_command_options.append("%s" % fastq_file)
    canu_command_options.append("2> %s" % os.path.join(sample_canu_dir, sample + ".stderr.log"))
    canu_command = ' '.join(canu_command_options)
    rerun_pipeline = subprocess.Popen(' && '.join([bam2fastq_command, cat_command, canu_command]), 
                                      shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = rerun_pipeline.communicate()
    print(stdout, stderr)
    contigFile = os.path.join(sample_canu_dir, canu_prefix+".contigs.fasta")
    if os.stat(contigFile).st_size == 0:
        # Failed to form contig, use tigInfoFile to get main contig out of unassembled file
        unassembled_fasta_file = os.path.join(sample_canu_dir, canu_prefix+".unassembled.fasta")
        # Open up unassembled fasta file:
        records = SeqIO.parse(unassembled_fasta_file, "fasta")
        # Open up tigInfo File
        tigInfoFile = os.path.join(sample_canu_dir, canu_prefix+".contigs.layout.tigInfo")
        tigInfoDF = pd.read_csv(tigInfoFile, sep="\t", header=0)
        # Get contigs with coverage > 1
        covered_contigs = ["tig{:08d}".format(contig)
                           for contig in tigInfoDF.loc[tigInfoDF["coverage"] > 1]["#tigID"].tolist()]
        # write contigs to contigFile
        #with open(contigFile, "w") as output_handle:
         #   SeqIO.write(record for, output_handle, "fasta")
        #for contig in covered_contigs:
        #    print(record_dict["{:08d}".format(contig)]))
        SeqIO.write([record for record in records if record.id in covered_contigs],
                    contigFile, "fasta")
    


unclassified.all.REF_JB2.circularised.bam
unclassified.all.REF_JB1.circularised.bam
unclassified.all.REF_YHC8.circularised.bam
unclassified.all.REF_PTS2.circularised.bam
unclassified.all.REF_PTS1.circularised.bam
unclassified.all.REF_YHC6.circularised.bam
unclassified.all.REF_YHC7.circularised.bam
unclassified.all.REF_YHC17.circularised.bam
('-- Canu release v1.5\nGFA alignments updated.\n', '[M::bam2fq_mainloop] discarded 0 singletons\n[M::bam2fq_mainloop] processed 181 reads\n')
('-- Canu release v1.5\nGFA alignments updated.\n', '[M::bam2fq_mainloop] discarded 0 singletons\n[M::bam2fq_mainloop] processed 431 reads\n')
('-- Canu release v1.5\nGFA alignments updated.\n', '[M::bam2fq_mainloop] discarded 0 singletons\n[M::bam2fq_mainloop] processed 1430 reads\n')
('-- Canu release v1.5\nGFA alignments updated.\n', '[M::bam2fq_mainloop] discarded 0 singletons\n[M::bam2fq_mainloop] processed 95 reads\n')
('-- Canu release v1.5\nGFA alignments updated.\n', '[M::bam2fq_mainloop] discarded 0

In [61]:
# Now run circulator to circularise the genome and generate a consensus
samples = genome_lengths.iterkeys()
circlator_dir = os.path.join(random_dir, "circlator")
if not os.path.isdir(circlator_dir):
    os.mkdir(circlator_dir)
for sample in samples:
    # Retrieve necessary files
    sample_canu_dir = os.path.join(random_dir, "canu_%s" % sample)
    contigFile = os.path.join(sample_canu_dir, "retry_canu_"+sample+".contigs.fasta")
    circlator_sample_dir = os.path.join(circlator_dir, sample)
    circlator_output_prefix = os.path.join(circlator_sample_dir, sample)
    # Output directory must not exist
    if os.path.isdir(circlator_sample_dir):
        shutil.rmtree(circlator_sample_dir)
    os.mkdir(circlator_sample_dir) 
    # Circlator command
    circlator_command_options = ["circlator minimus2"]
    circlator_command_options.append(contigFile)
    circlator_command_options.append(circlator_output_prefix)
    circlator_command = ' '.join(circlator_command_options)
    
    # Need to replace top line ">tig00000001.circularised" with actual name.
    # Can use sed command to do this.
    sed_command = "sed -i \"1 s/^.*$/>%s.circularised/\" %s" % (sample, circlator_output_prefix+".circularise.fasta")
    
    circlator_proc = subprocess.Popen(' && '.join([circlator_command, sed_command]), shell=True,
                                                   stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = circlator_proc.communicate()
    print(stdout, stderr)

('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')


In [62]:
# Create index and realign trimmed reads to the reference genome.
# Re-align reads using samtools
# Create alignment dir
samples = genome_lengths.iterkeys()
alignment_dir = os.path.join(random_dir, "alignment")
if not os.path.isdir(alignment_dir):
    os.mkdir(alignment_dir)
# Run alignment for each sample reads    
for sample in samples:
    # Get trimmed canu reads
    sample_canu_dir = os.path.join(random_dir, "canu_%s" % sample)
    sample_alignment_dir = os.path.join(alignment_dir, sample)
    if os.path.isdir(sample_alignment_dir):
        shutil.rmtree(sample_alignment_dir)
    os.mkdir(sample_alignment_dir)
    
    circlator_sample_dir = os.path.join(circlator_dir, sample)
    circlator_contig_file = os.path.join(circlator_sample_dir, sample+".circularise.fasta")

    sam_file = os.path.join(sample_alignment_dir, sample+".sam")
    bam_file = os.path.join(sample_alignment_dir, sample+".bam")
    bai_file = os.path.join(sample_alignment_dir, sample+".bai")

    if not os.path.isdir(sample_alignment_dir):
        os.mkdir(sample_alignment_dir)
    # Use the corrected and trimmed reads from Canu
    trimmed_reads = os.path.join(sample_canu_dir, "retry_canu_"+sample+".trimmedReads.fasta.gz")
    
    # Create the bwa and samtools indexes for the draft reference
    bwa_index_command = "bwa index %s" % circlator_contig_file
    samtools_index_command = "samtools faidx %s" % circlator_contig_file
    index_proc = subprocess.Popen(' && '.join([bwa_index_command, samtools_index_command]), shell=True,
                                  stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = index_proc.communicate()
    print(stdout, stderr)
    
    # Now run the bwa and sam2bam command
    bwa_align_command = "bwa mem -x ont2d %s %s > %s" % (circlator_contig_file, trimmed_reads, sam_file)
    
    # Samtools view and sort command
    samtools_view_and_sort_command = "samtools view -bS %s | samtools sort -o %s -" % (sam_file, bam_file)
    
    # Create bam index
    bam_index_command = "samtools index %s %s" % (bam_file, bai_file)
    
    # Run all commands sequentially
    alignment_commands = ' && '.join([bwa_align_command, samtools_view_and_sort_command,
                                       bam_index_command])
    alignment_proc = subprocess.Popen(alignment_commands, shell=True,
                                      stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = alignment_proc.communicate()
    print(stdout, stderr)

('', '[bwa_index] Pack FASTA... 0.00 sec\n[bwa_index] Construct BWT for the packed sequence...\n[bwa_index] 0.00 seconds elapse.\n[bwa_index] Update BWT... 0.00 sec\n[bwa_index] Pack forward-only FASTA... 0.01 sec\n[bwa_index] Construct SA from BWT and Occ... 0.00 sec\n[main] Version: 0.7.15-r1140\n[main] CMD: bwa index /data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED/unclassified/circlator/YHC8/YHC8.circularise.fasta\n[main] Real time: 0.045 sec; CPU: 0.023 sec\n')
('', '[M::bwa_idx_load_from_disk] read 0 ALT contigs\n[M::process] read 113 sequences (707415 bp)...\n[M::mem_process_seqs] Processed 113 reads in 6.151 CPU sec, 6.150 real sec\n[main] Version: 0.7.15-r1140\n[main] CMD: bwa mem -x ont2d /data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED/unclassified/circlator/YHC8/YHC8.circularise.fasta /data/Bioinfo/bioinfo-proj-alexis/2017_07_09_PLASMIDS_BARCODED/unclassified/canu_YHC8/retry_canu_YHC8.trimmedReads.fasta.gz\n[main] Real time: 6.172 sec; CPU: 6.171

In [63]:
# Test for circularisation, export to TSV
samples = genome_lengths.iterkeys()
circlator_dir = os.path.join(random_dir, "circlator")
circularised_df = pd.DataFrame(columns=["sample", "genome_size", "circularised"])
tsv_output_file = os.path.join(PRESENTATION_DIR, "data", "using_unclassified_data_genome_status.tsv")
for sample in samples:
    circlator_sample_dir = os.path.join(circlator_dir, sample)
    circlator_log_file = os.path.join(circlator_sample_dir, sample+".log")
    # Example line in log file: tig00000001 circularised: True
    is_circularised_command = "grep circularised: %s | rev | cut -d' ' -f1 | rev" % circlator_log_file
    is_circularised_proc = subprocess.Popen(is_circularised_command, shell=True,
                                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = is_circularised_proc.communicate()
    is_circularised = stdout.rstrip()=="True"
    print(is_circularised)
    circularised_as_seres = pd.Series(data=[sample, genome_lengths[sample], is_circularised], 
                                      index=["sample", "genome_size", "circularised"])
    circularised_df = circularised_df.append(circularised_as_seres, ignore_index=True)
circularised_df.to_csv(tsv_output_file, index=False, sep="\t")
print(circularised_df)

True
True
True
True
True
False
True
False
  sample genome_size circularised
0   YHC8       27458         True
1    JB2       12228         True
2    JB1        8894         True
3   PTS1       13987         True
4   PTS2       13553         True
5   YHC6       20112        False
6   YHC7       15924         True
7  YHC17       21705        False
